### Hypothesis Testing

Null Hypothesis: There is no difference in avg. goals/game between EPL and Bundesliga

Alternative hypthesis: The English Premier League has a higher avg. goals/game ratio than the Bundesliga

We used a one-tailed T-test because we did not know population means or standard devs, and were comparing if one variable was higher than another

In [1]:
import pymongo
import pandas as pd
import numpy as np
import json
import requests
import sqlite3
import scipy.stats as stats

cur = sqlite3.connect('database.sqlite').cursor()
from server import *

In [9]:
def get_matches_w_tots(season):
    # get DF of league
    wanted_matches = get_season_games(season)
    
    # append column names
    wanted_matches.columns = [x[0] for x in cur.description]
    
    # get match totals
    match_totals = []
    num_matches = len(wanted_matches)
    for i in range(num_matches):
        match_totals.append(wanted_matches.iloc[i]['FTHG'] + wanted_matches.iloc[i]['FTAG'])
        
    # append match totals to wanted_matches df
    wanted_matches['match_tots'] = match_totals
    wanted_matches = wanted_matches.groupby('league')
    return wanted_matches

In [3]:
# return a dataframe of descriptive statistics for matches in the provided season for EPL and Bundesliga
def get_description(season):
    return get_matches_w_tots(season).describe()['match_tots']

In [4]:
df = get_description(2011)
df

,count,mean,std,min,25%,50%,75%,max
league,,,,,,,,
D1,306.0,2.859477,1.678170,0.0,2.0,3.0,4.0,8.0
E0,380.0,2.805263,1.687756,0.0,2.0,3.0,4.0,10.0


In [5]:
# get t statistic and p value from EPL and Bundesliga
def t_test(df):
    results = stats.ttest_ind_from_stats(df['mean']['E0'], df['std']['E0'], df['count']['E0'], 
                           df['mean']['D1'], df['std']['D1'],df['count']['D1'])
    t = results[0]
    p = results[1]/2
    return t, p

In [8]:
t, p = t_test(get_description(2011))
print("t: ", t, "p: ", p)

t:  -0.41926780962255317 p:  0.33757602862256464


### In conclusion...
Between 2005-2015 , the Bundesliga has an average of 0.2 more goals scored per game (p < 0.01) than the English Premier League.  Therefor, although we cannot reject our null hypothesis, we can recommend purchasing the broadcast rights to the German Bundesliga over the EPL.

In [14]:
import scipy.stats as sc
games = get_season_games(2005)
for i in range(2006, 2016):
    games = pd.concat([games, get_season_games(i)])
grouped = games.groupby('league')
means = grouped.mean()
means = dict(means['FTHG'] + means['FTAG'])
std = grouped.std()
std = dict(std['FTHG'] + std['FTAG'])
counts = dict(grouped.count()['FTAG'])
sc.stats.ttest_ind_from_stats(means['D1'], std['D1'], counts['D1'],
                              means['E0'], std['E0'], counts['E0'])

Ttest_indResult(statistic=3.690275139802319, pvalue=0.00022558963053935123)